In [ ]:
# !pip install metatrader5

In [ ]:
#!pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
# Straddle with Hedging

In [ ]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["XAUUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "EURUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 7, 18, 12, 40, 0),  # First timer
    #datetime(2024, 7, 3, 14, 58, 0),  # Second timer
    #datetime(2024, 7, 3, 18, 58, 0)   # Third timer
]

# Define the lot size and pip parameters
initial_lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}
hedging_distance_pip = 40
trailing_pip = 15
trail_amount_update = 3
default_sl_pip = 10

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the initial orders
def place_initial_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_price_buy = ask_price + 20 * pip
    stop_price_sell = bid_price - 20 * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")


# Function to trail stop loss
def trail_sl(symbol):
    pip = pip_dict[symbol]
    MAX_DIST_SL = trailing_pip * pip
    TRAIL_AMOUNT = trail_amount_update * pip
    DEFAULT_SL = default_sl_pip * pip

    positions = mt5.positions_get(symbol=symbol)
    if not positions:
        return

    for position in positions:
        order_type = position.type
        price_current = mt5.symbol_info_tick(symbol).ask if order_type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).bid
        price_open = position.price_open
        sl = position.sl

        dist_from_sl = abs(price_current - sl)

        if dist_from_sl > MAX_DIST_SL:
            if sl != 0.0:
                new_sl = sl + TRAIL_AMOUNT if order_type == mt5.ORDER_TYPE_BUY else sl - TRAIL_AMOUNT
            else:
                new_sl = price_open - DEFAULT_SL if order_type == mt5.ORDER_TYPE_BUY else price_open + DEFAULT_SL

            request = {
                'action': mt5.TRADE_ACTION_SLTP,
                'position': position.ticket,
                'sl': new_sl,
            }

            result = mt5.order_send(request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to modify SL for {symbol} position {position.ticket}, retcode={result.retcode}")
                print(f"Error: {result}")
            else:
                print(f"Modified SL for {symbol} position {position.ticket}, new SL={new_sl}")


# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol):
    while True:
        trail_sl(symbol)
        time.sleep(1)


# Function to monitor for triggered orders and handle hedging
def monitor_orders(symbol):
    while True:
        positions = mt5.positions_get(symbol=symbol)
        orders = mt5.orders_get(symbol=symbol)

        if positions:
            for position in positions:
                if position.type == mt5.ORDER_TYPE_BUY:
                    print(f"Buy position detected for {symbol}, deleting sell stop and placing hedging order")
                    delete_opposite_order(symbol, mt5.ORDER_TYPE_SELL_STOP)
                    place_hedging_order(symbol, mt5.ORDER_TYPE_SELL, position.price_open + hedging_distance_pip * pip_dict[symbol])
                elif position.type == mt5.ORDER_TYPE_SELL:
                    print(f"Sell position detected for {symbol}, deleting buy stop and placing hedging order")
                    delete_opposite_order(symbol, mt5.ORDER_TYPE_BUY_STOP)
                    place_hedging_order(symbol, mt5.ORDER_TYPE_BUY, position.price_open - hedging_distance_pip * pip_dict[symbol])
                
        monitor_trailing_stops(symbol)
        time.sleep(1)

# Function to delete the opposite order
def delete_opposite_order(symbol, order_type):
    orders = mt5.orders_get(symbol=symbol)
    if orders:
        for order in orders:
            if order.type == order_type:
                cancel_request = {
                    "action": mt5.TRADE_ACTION_REMOVE,
                    "order": order.ticket,
                }
                result = mt5.order_send(cancel_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Failed to cancel order {order.ticket} for {symbol}, retcode={result.retcode}")
                else:
                    print(f"Order {order.ticket} for {symbol} canceled successfully.")

# Function to place a hedging order
def place_hedging_order(symbol, order_type, price):
    hedging_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot,
        "type": order_type,
        "price": price,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script hedging order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    result = mt5.order_send(hedging_request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Hedging order for {symbol} failed, retcode={result.retcode}")
        print(f"Error: {result}")
    else:
        print(f"Hedging order for {symbol} placed successfully, ticket={result.order}")

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        print(f"Countdown to {scheduled_time}")
        display_countdown(scheduled_time)

        for symbol in symbols:
            place_initial_orders(symbol)
            monitor_orders(symbol)

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


Countdown to 2024-07-18 12:40:00
Buy stop order for XAUUSD placed successfully, ticket=90128451
Sell stop order for XAUUSD placed successfully, ticket=90128453
Modified SL for XAUUSD position 90128451, new SL=2466.27
Failed to modify SL for XAUUSD position 90128453, retcode=10016
Error: OrderSendResult(retcode=10016, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid stops', request_id=2492034111, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=2463.95, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=90128453, position_by=0))
Modified SL for XAUUSD position 90128453, new SL=2463.95


In [ ]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 7, 15, 22, 55, 0),  # First timer
    # datetime(2024, 7, 3, 14, 58, 0),  # Second timer
    # datetime(2024, 7, 3, 18, 58, 0)   # Third timer
]

# Define the lot size and pip parameters
initial_lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}
trailing_pip = 15
trail_amount_update = 3
default_sl_pip = 10
bs_stop = 15
stop_loss = 1000
hedging_distance_pip = 40

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the initial orders
def place_initial_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss * pip
    stop_price_buy = ask_price + bs_stop * pip

    stop_loss_price_sell = bid_price + stop_loss * pip
    stop_price_sell = bid_price - bs_stop * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        #"sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        #"sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to delete the opposite order
def delete_opposite_order(symbol, order_type):
    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders found for {symbol}")
        return

    for order in orders:
        if order.type == order_type:
            delete_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel opposite order",
            }
            result = mt5.order_send(delete_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to delete order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} deleted successfully.")

# Function to place a hedging order
def place_hedging_order(symbol, order_type, price):
    request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot,
        "type": order_type,
        "price": price,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script hedging order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Hedging order for {symbol} failed, retcode={result.retcode}")
        print(f"Error: {result}")
    else:
        print(f"Hedging order for {symbol} placed successfully, ticket={result.order}")

# Function to trail stop loss
def trail_sl(symbol):
    pip = pip_dict[symbol]
    MAX_DIST_SL = trailing_pip * pip
    TRAIL_AMOUNT = trail_amount_update * pip
    DEFAULT_SL = default_sl_pip * pip

    positions = mt5.positions_get(symbol=symbol)
    if not positions:
        return

    for position in positions:
        order_type = position.type
        price_current = mt5.symbol_info_tick(symbol).ask if order_type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).bid
        price_open = position.price_open
        sl = position.sl

        dist_from_sl = abs(price_current - sl)

        if dist_from_sl > MAX_DIST_SL:
            if sl != 0.0:
                new_sl = sl + TRAIL_AMOUNT if order_type == mt5.ORDER_TYPE_BUY else sl - TRAIL_AMOUNT
            else:
                new_sl = price_open - DEFAULT_SL if order_type == mt5.ORDER_TYPE_BUY else price_open + DEFAULT_SL

            request = {
                'action': mt5.TRADE_ACTION_SLTP,
                'position': position.ticket,
                'sl': new_sl,
            }

            result = mt5.order_send(request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to modify SL for {symbol} position {position.ticket}, retcode={result.retcode}")
                print(f"Error: {result}")
            else:
                print(f"Modified SL for {symbol} position {position.ticket}, new SL={new_sl}")

# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol):
    while True:
        trail_sl(symbol)
        time.sleep(1)

# Function to monitor for triggered orders and handle hedging
def monitor_orders(symbol):
    while True:
        positions = mt5.positions_get(symbol=symbol)
        orders = mt5.orders_get(symbol=symbol)

        if positions:
            for position in positions:
                if position.type == mt5.ORDER_TYPE_BUY:
                    print(f"Buy position detected for {symbol}, deleting sell stop and placing hedging order")
                    delete_opposite_order(symbol, mt5.ORDER_TYPE_SELL_STOP)
                    place_hedging_order(symbol, mt5.ORDER_TYPE_SELL, position.price_open - hedging_distance_pip * pip_dict[symbol])
                elif position.type == mt5.ORDER_TYPE_SELL:
                    print(f"Sell position detected for {symbol}, deleting buy stop and placing hedging order")
                    delete_opposite_order(symbol, mt5.ORDER_TYPE_BUY_STOP)
                    place_hedging_order(symbol, mt5.ORDER_TYPE_BUY, position.price_open + hedging_distance_pip * pip_dict[symbol])
                
        monitor_trailing_stops(symbol)
        time.sleep(1)

# Function to schedule the initial orders
def schedule_initial_orders():
    while True:
        current_time = datetime.now()
        for scheduled_time in scheduled_times:
            if scheduled_time <= current_time < scheduled_time + timedelta(seconds=10):
                for symbol in symbols:
                    place_initial_orders(symbol)
                scheduled_times.remove(scheduled_time)
        time.sleep(1)

# Run the schedule in a separate thread to avoid blocking
import threading
threading.Thread(target=schedule_initial_orders).start()

# Monitor orders and adjust trailing stops for each symbol
for symbol in symbols:
    threading.Thread(target=monitor_orders, args=(symbol,)).start()


In [ ]:
# Fixing attempt at placing hedging trades price

In [ ]:
# Straddle and zonal hedging first try

In [ ]:
import MetaTrader5 as mt5
import time

# Connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# Define the symbols and initial lot size
symbol = "EURUSD"
initial_lot = 0.01

# Define the initial buystop and sellstop prices
buystop_price = 1.2000
sellstop_price = 1.1980

# Define the pip distance for new trades
pip_distance = 20 * 0.0001  # 20 pips in decimal form for a 5-digit broker

# Define the lot size multiplier
lot_multiplier = 1.40

def place_order(action, lot, price):
    order_type = mt5.ORDER_TYPE_BUY_STOP if action == "buy" else mt5.ORDER_TYPE_SELL_STOP
    request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 10,
        "magic": 234000,
        "comment": "Straddle and Zonal Hedging",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    return result

def cancel_pending_orders():
    orders = mt5.orders_get(symbol=symbol)
    for order in orders:
        if order.symbol == symbol:
            mt5.order_delete(order.ticket)

# Place initial buystop and sellstop orders
place_order("buy", initial_lot, buystop_price)
place_order("sell", initial_lot, sellstop_price)

# Monitor for triggers
while True:
    positions = mt5.positions_get(symbol=symbol)
    if positions:
        break
    time.sleep(1)

# Determine which order was triggered
triggered_position = positions[0]
triggered_price = triggered_position.price_open
triggered_type = "buy" if triggered_position.type == mt5.ORDER_TYPE_BUY else "sell"

# Cancel the opposite pending order
cancel_pending_orders()

# Set up zonal hedging
if triggered_type == "buy":
    next_price = triggered_price - pip_distance
    place_order("sell", initial_lot * lot_multiplier, next_price)
    place_order("buy", initial_lot * (lot_multiplier**2 - 1), triggered_price)
    place_order("sell", initial_lot * lot_multiplier * (lot_multiplier**2 - 1), next_price)
else:
    next_price = triggered_price + pip_distance
    place_order("buy", initial_lot * lot_multiplier, next_price)
    place_order("sell", initial_lot * (lot_multiplier**2 - 1), triggered_price)
    place_order("buy", initial_lot * lot_multiplier * (lot_multiplier**2 - 1), next_price)

# Close connection
mt5.shutdown()
